<a href="https://colab.research.google.com/github/vignejs/cover-song-identification/blob/master/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [39]:
!pip install tensorflow-addons==0.9.1

In [40]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

In [41]:
feature_type = "crema"

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    root = Path("/content/drive/My Drive/cosoid")
else:
    root = Path.cwd()

testpath = root / 'datasets' / 'testset.h5'
modelpath = root / "weights" / "weights.best-9.22e+06.hdf5"

In [42]:
df = pd.HDFStore(testpath).select(feature_type).sort_index(level=0)
df

0         1    ...       498       499
work_id track_id chroma_bins                      ...                    
W1004   W1004-P0 0            0.061550  0.068997  ...  0.092226  0.225978
                 1            0.011360  0.009081  ...  0.037040  0.070564
                 2            0.164952  0.120212  ...  0.309661  0.072964
                 3            0.026743  0.026783  ...  0.031216  0.026514
                 4            0.012861  0.009130  ...  0.002354  0.079537
...                                ...       ...  ...       ...       ...
W922    W922-P9  7            0.051805  0.049097  ...  0.004867  0.008980
                 8            0.034500  0.030481  ...  0.209956  0.578368
                 9            0.139943  0.145197  ...  0.945642  0.627132
                 10           0.002773  0.003158  ...  0.000930  0.001695
                 11           0.125918  0.139058  ...  0.068489  0.245725

[4560 rows x 500 columns]

In [43]:
tracks = df.index.droplevel('chroma_bins').drop_duplicates().to_frame().reset_index(drop=True)
tracks

,work_id,track_id
0,W1004,W1004-P0
1,W1004,W1004-P1
2,W1004,W1004-P10
3,W1004,W1004-P12
4,W1004,W1004-P14
...,...,...
375,W922,W922-P5
376,W922,W922-P6
377,W922,W922-P7
378,W922,W922-P8


In [44]:
countdf = tracks.groupby('work_id').count()
countdf

,track_id
work_id,
W1004,33
W1009,29
W348,29
W350,38
W369,64
W487,44
W63,31
W794,15
W81,64


In [45]:
query_tracks = tracks.groupby('work_id').first()
query_tracks

,track_id
work_id,
W1004,W1004-P0
W1009,W1009-P0
W348,W348-P1
W350,W350-P0
W369,W369-P0
W487,W487-P0
W63,W63-P1
W794,W794-P12
W81,W81-P1


In [46]:
querydf = df[np.in1d(df.index.get_level_values('track_id'), query_tracks.values)]
querydf

0         1    ...       498       499
work_id track_id chroma_bins                      ...                    
W1004   W1004-P0 0            0.061550  0.068997  ...  0.092226  0.225978
                 1            0.011360  0.009081  ...  0.037040  0.070564
                 2            0.164952  0.120212  ...  0.309661  0.072964
                 3            0.026743  0.026783  ...  0.031216  0.026514
                 4            0.012861  0.009130  ...  0.002354  0.079537
...                                ...       ...  ...       ...       ...
W922    W922-P0  7            0.013857  0.012236  ...  0.004932  0.012000
                 8            0.011654  0.010796  ...  0.979914  0.914290
                 9            0.203621  0.195214  ...  0.007311  0.024320
                 10           0.004963  0.004373  ...  0.001542  0.004056
                 11           0.050657  0.033000  ...  0.989206  0.939168

[120 rows x 500 columns]

In [47]:
testdf = df[~np.in1d(df.index.get_level_values('track_id'), query_tracks.values)]
testdf

0         1    ...       498       499
work_id track_id chroma_bins                      ...                    
W1004   W1004-P1 0            0.190049  0.253458  ...  0.884587  0.564905
                 1            0.040592  0.088572  ...  0.041143  0.071651
                 2            0.011218  0.007471  ...  0.010354  0.030843
                 3            0.077429  0.050899  ...  0.913755  0.662081
                 4            0.006846  0.004876  ...  0.003852  0.008740
...                                ...       ...  ...       ...       ...
W922    W922-P9  7            0.051805  0.049097  ...  0.004867  0.008980
                 8            0.034500  0.030481  ...  0.209956  0.578368
                 9            0.139943  0.145197  ...  0.945642  0.627132
                 10           0.002773  0.003158  ...  0.000930  0.001695
                 11           0.125918  0.139058  ...  0.068489  0.245725

[4440 rows x 500 columns]

In [48]:
len_query = querydf.shape[0] // 12
len_test = testdf.shape[0] // 12

query_arr = querydf.values.reshape(len_query, 12, 500, 1)
test_arr = testdf.values.reshape(len_test, 12, 500, 1)

query_arr = np.tile(query_arr, (1, 2, 1, 1))[:, :-1, :, :]
test_arr = np.tile(test_arr, (1, 2, 1, 1))[:, :-1, :, :]

In [49]:
# Initializers
def W_init(shape, dtype=None):
    """Initialize weights"""
    values = np.random.normal(loc=0, scale=1e-2, size=shape)
    return K.variable(values, dtype=dtype)

def W_init_dense(shape, dtype=None):
    """Initialize weights"""
    values = np.random.normal(loc=0, scale=2e-1, size=shape)
    return K.variable(values, dtype=dtype)
 
def b_init(shape, dtype=None):
    """Initialize bias"""
    values = np.random.normal(loc=0.5, scale=1e-2, size=shape)
    return K.variable(values, dtype=dtype)

In [50]:
custom_objects = {"W_init": W_init,
                  "W_init_dense": W_init_dense,
                  "b_init": b_init}

model = load_model(modelpath, custom_objects=custom_objects)

In [51]:
model.layers

In [52]:
# get embeddings
embed_model = model.layers[2]
query_embeddings = embed_model.predict(query_arr, verbose=1, batch_size=12)
test_embeddings = embed_model.predict(test_arr, verbose=1, batch_size=120)

4/4 [==============================] - 1s 353ms/step


In [53]:
dense_layer = model.layers[-1]
pairwise_distance = np.abs(query_embeddings[:, None, :] - test_embeddings)
pairwise_predictions = dense_layer(pairwise_distance.reshape(-1, 512)).numpy()
ypred = pairwise_predictions.reshape(len_query, len_test)

In [54]:
count = np.ravel(countdf.values - 1)
ones = list(map(np.ones, count))
from scipy.linalg import block_diag
ytrue = block_diag(*ones)

In [55]:
k = len_test
spred = (ypred).argsort(axis=1)[:, :k]
found = np.take_along_axis(ytrue, spred, axis=1)
top1 = np.mean(found[:, :1])
top2 = np.mean(found[:, :2])
top5 = np.mean(found[:, :5])
top10 = np.mean(found[:, :10])
pos = np.arange(1, spred.shape[1]+1)
pos = np.expand_dims(pos, axis=0)
prec = np.cumsum(found, axis=1) / pos
mask = (found > 0).astype(np.float32)
ap = np.sum(prec * mask, axis=1) / np.minimum(k, np.sum(ytrue, axis=1))
mAP =  np.mean(ap)
print('mAP: {:.3f}'.format(mAP))
print('Top1: {:.3f}'.format(top1))
print('Top2: {:.3f}'.format(top2))
print('Top5: {:.3f}'.format(top5))
print('Top10: {:.3f}'.format(top10))

mAP: 0.626
Top1: 0.700
Top2: 0.800
Top5: 0.740
Top10: 0.750


In [124]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=3)

X = np.r_[query_embeddings, test_embeddings]
X = pca.fit_transform(X)

In [125]:
query_labels = querydf.index.droplevel('chroma_bins').drop_duplicates().get_level_values('work_id').values
test_labels = testdf.index.droplevel('chroma_bins').drop_duplicates().get_level_values('work_id').values
labels = np.r_[query_labels, test_labels]

colors = ['#e94560', '#1a1a2e', '#213e3b', '#a6f6f1', '#7579e7', 
          '#edc988', '#8db596', '#a37eba', '#f4abc4', '#463333']

color_map = {label: color for label, color in zip(np.unique(labels), colors)}     

color = [color_map[i] for i in labels]

In [126]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=X[:, 0],
    y=X[:, 1],
    z=X[:, 2],
    mode='markers',
    text=labels,
    hoverinfo='text',
    marker=dict(
        size=12,
        color=color,
        opacity=0.8
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()